Setup (imports, paths, seed)

In [24]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import numpy as np
import pandas as pd

from src.config import get_paths, set_seed
from src.io import read_table, save_table

from src.stats import (
    spearman_with_bootstrap_ci,
    icc2_absolute,
    kruskal_wallis_epsilon2,
    dunn_posthoc_holm,
)

paths = get_paths()
set_seed(42)

LABELS_ORDER = ["GC", "G2", "G5", "G7", "G14"]


Load datasets (ERAT outputs + EHAT dataset)

In [25]:
df_erat = pd.read_csv(paths.processed / "erat_outputs.csv")
df_ehat = pd.read_csv(paths.processed / "ehat_dataset.csv")

print("Loaded:")
print("df_erat:", df_erat.shape, "| cols:", df_erat.columns.tolist())
print("df_ehat:", df_ehat.shape, "| cols:", df_ehat.columns.tolist())


Loaded:
df_erat: (571, 4) | cols: ['days_true', 'erat_days_pred', 'erat_norm', 'group']
df_ehat: (571, 5) | cols: ['pct_contractile_area', 'pct_contractile_area_norm', 'ehat_norm', 'group', 'days_post_btx']


Build df_compare

In [26]:
# Basic alignment checks
assert len(df_erat) == len(df_ehat), "df_erat and df_ehat must have same number of rows (image-level)."

# Check group/days consistency
assert (df_erat["group"].values == df_ehat["group"].values).all(), "Group mismatch between ERAT and EHAT datasets."
assert (df_erat["days_true"].values == df_ehat["days_post_btx"].values).all(), "Days mismatch between ERAT and EHAT datasets."

df_compare = pd.DataFrame({
    "group": df_ehat["group"].values,
    "days_post_btx": df_ehat["days_post_btx"].values,
    "erat_days_pred": df_erat["erat_days_pred"].values,
    "erat_norm": df_erat["erat_norm"].values,
    "ehat_norm": df_ehat["ehat_norm"].values,
    "pct_contractile_area": df_ehat["pct_contractile_area"].values,
    "pct_contractile_area_norm": df_ehat["pct_contractile_area_norm"].values,
})

display(df_compare.head())
print(df_compare.shape)

,group,days_post_btx,erat_days_pred,erat_norm,ehat_norm,pct_contractile_area,pct_contractile_area_norm
0,GC,0,-0.110488,0.029865,0.391910,0.484456,0.608090
1,GC,0,-0.021282,0.035919,0.505023,0.407464,0.494977
2,GC,0,-0.073871,0.032350,0.568629,0.364170,0.431371
3,GC,0,0.118741,0.045421,0.204136,0.612266,0.795864
4,GC,0,-0.276701,0.018586,0.109739,0.676517,0.890261


(571, 7)


In [27]:
save_table(df_compare, paths.processed / "df_compare_erat_ehat.csv")

WindowsPath('c:/Users/modre/Documents/masseter/data/processed/df_compare_erat_ehat.csv')

Spearman (ERAT_norm vs EHAT_norm)

In [28]:
res = spearman_with_bootstrap_ci(
    x=df_compare["erat_norm"].values,
    y=df_compare["ehat_norm"].values,
    n_boot=2000,
    ci=0.95,
    seed=42,
)

df_spearman = pd.DataFrame([{
    "metric": "Spearman_rho",
    "value": res["rho"],
    "p_value": res["p_value"],
    "ci_2p5": res["ci_low"],
    "ci_97p5": res["ci_high"],
    "n_samples": res["n"],
}])

display(df_spearman)
save_table(df_spearman, paths.results / "tables" / "erat_vs_ehat_spearman.csv")

,metric,value,p_value,ci_2p5,ci_97p5,n_samples
0,Spearman_rho,0.237711,8.901967e-09,0.150256,0.315581,571


WindowsPath('c:/Users/modre/Documents/masseter/results/tables/erat_vs_ehat_spearman.csv')

ICC(2,1) absolute agreement

In [29]:
df_long = pd.DataFrame({
    "target": np.repeat(np.arange(len(df_compare)), 2),
    "rater": np.tile(["ERAT", "EHAT"], len(df_compare)),
    "rating": np.concatenate([df_compare["erat_norm"].values, df_compare["ehat_norm"].values]),
})

icc = icc2_absolute(df_long, targets="target", raters="rater", ratings="rating")

# Keep only ICC2 row (principal)
icc2 = icc.loc[icc["Type"] == "ICC2"].copy()
display(icc2)

save_table(icc,  paths.results / "tables" / "erat_vs_ehat_icc_all.csv")
save_table(icc2, paths.results / "tables" / "erat_vs_ehat_icc2.csv")

,Type,Description,ICC,F,df1,df2,pval,CI95%
1,ICC2,Single random raters,0.846738,12.05103,570,570,1.138105e-158,"[0.82, 0.87]"


WindowsPath('c:/Users/modre/Documents/masseter/results/tables/erat_vs_ehat_icc2.csv')

Bland–Altman

In [30]:
diff = df_compare["erat_norm"].values - df_compare["ehat_norm"].values
mean_ = (df_compare["erat_norm"].values + df_compare["ehat_norm"].values) / 2

bias = float(np.mean(diff))
sd = float(np.std(diff, ddof=1))
loa_low = bias - 1.96 * sd
loa_high = bias + 1.96 * sd

df_ba = pd.DataFrame([{
    "bias": bias,
    "sd_diff": sd,
    "loa_low": loa_low,
    "loa_high": loa_high,
    "n_samples": len(diff),
}])

display(df_ba)
save_table(df_ba, paths.results / "tables" / "erat_vs_ehat_bland_altman_stats.csv")


,bias,sd_diff,loa_low,loa_high,n_samples
0,-0.00414,0.324832,-0.64081,0.63253,571


WindowsPath('c:/Users/modre/Documents/masseter/results/tables/erat_vs_ehat_bland_altman_stats.csv')

In [31]:
try:
    import matplotlib.pyplot as plt

    fig = plt.figure(figsize=(8, 5))
    plt.scatter(mean_, diff, alpha=0.6)
    plt.axhline(bias, linestyle="--")
    plt.axhline(loa_low, linestyle="--")
    plt.axhline(loa_high, linestyle="--")
    plt.xlabel("Mean (ERAT_norm + EHAT_norm) / 2")
    plt.ylabel("Difference (ERAT_norm - EHAT_norm)")
    plt.title("Bland–Altman: ERAT_norm vs EHAT_norm")

    fig_path = paths.results / "figures" / "bland_altman_erat_vs_ehat.png"
    fig_path.parent.mkdir(parents=True, exist_ok=True)
    plt.savefig(fig_path, dpi=300, bbox_inches="tight")
    plt.close(fig)
    print("Saved:", fig_path)
except Exception as e:
    print("Skipping Bland–Altman plot (matplotlib unavailable):", e)


Saved: c:\Users\modre\Documents\masseter\results\figures\bland_altman_erat_vs_ehat.png


Temporal trajectories (mean ± SD by group)

In [32]:
def mean_sd_ci95(x: pd.Series) -> dict:
    x = x.dropna().astype(float)
    n = len(x)
    mean = float(x.mean())
    sd = float(x.std(ddof=1)) if n > 1 else 0.0
    se = sd / np.sqrt(n) if n > 1 else 0.0
    ci_low = mean - 1.96 * se
    ci_high = mean + 1.96 * se
    return {"n": n, "mean": mean, "sd": sd, "ci_low": ci_low, "ci_high": ci_high}

rows = []
for g in LABELS_ORDER:
    sub = df_compare[df_compare["group"] == g]
    e = mean_sd_ci95(sub["erat_norm"])
    h = mean_sd_ci95(sub["ehat_norm"])
    rows.append({
        "group": g,
        "days_post_btx": int(sub["days_post_btx"].iloc[0]),
        "n_images": int(len(sub)),
        "ERAT_mean": e["mean"], "ERAT_sd": e["sd"], "ERAT_ci95_low": e["ci_low"], "ERAT_ci95_high": e["ci_high"],
        "EHAT_mean": h["mean"], "EHAT_sd": h["sd"], "EHAT_ci95_low": h["ci_low"], "EHAT_ci95_high": h["ci_high"],
    })

table5 = pd.DataFrame(rows).sort_values("days_post_btx")
display(table5)

save_table(table5, paths.results / "tables" / "table5_erat_vs_ehat_by_group.csv")


,group,days_post_btx,n_images,ERAT_mean,ERAT_sd,ERAT_ci95_low,ERAT_ci95_high,EHAT_mean,EHAT_sd,EHAT_ci95_low,EHAT_ci95_high
0,GC,0,120,0.086011,0.083700,0.071035,0.100987,0.345884,0.141317,0.320599,0.371169
1,G2,2,110,0.202843,0.056748,0.192238,0.213448,0.421234,0.164430,0.390506,0.451963
2,G5,5,121,0.358743,0.054804,0.348978,0.368508,0.422975,0.163122,0.393909,0.452040
3,G7,7,110,0.444586,0.076605,0.430270,0.458902,0.414101,0.148866,0.386281,0.441921
4,G14,14,110,0.986472,0.014916,0.983684,0.989259,0.465905,0.172134,0.433737,0.498073


WindowsPath('c:/Users/modre/Documents/masseter/results/tables/table5_erat_vs_ehat_by_group.csv')

In [33]:
try:
    import matplotlib.pyplot as plt

    fig = plt.figure(figsize=(8, 5))

    x = table5["group"].tolist()
    plt.errorbar(x, table5["ERAT_mean"], yerr=table5["ERAT_sd"], marker="o", capsize=4, label="ERAT_norm (LightGBM)")
    plt.errorbar(x, table5["EHAT_mean"], yerr=table5["EHAT_sd"], marker="s", capsize=4, label="EHAT_norm (Histology)")

    plt.ylim(-0.05, 1.05)
    plt.xlabel("Experimental group (days post-BoNT-A)")
    plt.ylabel("Normalized atrophy score")
    plt.title("Temporal trajectories of ERAT_norm and EHAT_norm")
    plt.legend()

    fig_path = paths.results / "figures" / "temporal_trajectories_erat_ehat.png"
    fig_path.parent.mkdir(parents=True, exist_ok=True)
    plt.savefig(fig_path, dpi=300, bbox_inches="tight")
    plt.close(fig)
    print("Saved:", fig_path)

except Exception as e:
    print("Skipping trajectory plot (matplotlib unavailable):", e)


Saved: c:\Users\modre\Documents\masseter\results\figures\temporal_trajectories_erat_ehat.png


Kruskal–Wallis + Dunn–Holm (ERAT_norm and EHAT_norm)

In [34]:
from src.stats import kruskal_dunn_holm

# ERAT_norm
kw_erat, dunn_erat = kruskal_dunn_holm(
    df_compare,
    value_col="erat_norm",
    group_col="group",
    group_order=LABELS_ORDER,
)

# EHAT_norm
kw_ehat, dunn_ehat = kruskal_dunn_holm(
    df_compare,
    value_col="ehat_norm",
    group_col="group",
    group_order=LABELS_ORDER,
)

display(kw_erat)
display(dunn_erat)

display(kw_ehat)
display(dunn_ehat)

save_table(kw_erat,   paths.results / "tables" / "erat_kw_summary.csv")
save_table(dunn_erat, paths.results / "tables" / "erat_dunn_holm.csv")

save_table(kw_ehat,   paths.results / "tables" / "ehat_kw_summary.csv")
save_table(dunn_ehat, paths.results / "tables" / "ehat_dunn_holm.csv")

,metric,value
0,H,5.035712e+02
1,p_value,1.131480e-107
2,epsilon2,8.826347e-01
3,n_total,5.710000e+02
4,k_groups,5.000000e+00


,GC,G2,G5,G7,G14
GC,1.000000e+00,6.603775e-05,9.088981e-26,7.166411e-44,1.303646e-89
G2,6.603775e-05,1.000000e+00,1.337293e-09,1.841760e-21,1.434535e-54
G5,9.088981e-26,1.337293e-09,1.000000e+00,2.885519e-04,7.906503e-22
G7,7.166411e-44,1.841760e-21,2.885519e-04,1.000000e+00,5.326264e-09
G14,1.303646e-89,1.434535e-54,7.906503e-22,5.326264e-09,1.000000e+00


,metric,value
0,H,5.041776e+01
1,p_value,2.953735e-10
2,epsilon2,8.201017e-02
3,n_total,5.710000e+02
4,k_groups,5.000000e+00


,GC,G2,G5,G7,G14
GC,1.000000e+00,0.000776,0.000041,0.015251,5.138563e-11
G2,7.760898e-04,1.000000,0.778362,0.778362,1.638398e-02
G5,4.118798e-05,0.778362,1.000000,0.434631,5.986391e-02
G7,1.525062e-02,0.778362,0.434631,1.000000,1.006325e-03
G14,5.138563e-11,0.016384,0.059864,0.001006,1.000000e+00


WindowsPath('c:/Users/modre/Documents/masseter/results/tables/ehat_dunn_holm.csv')